In [1]:
def ttest_experiment_seg(experiment, length = 0):
   #variables we need 
    fixed_traces = []
    random_traces = []
    welsh_t = []
    new_mr = []
    new_mf =[]
    new_sf = []
    new_sr = []
    mf_old = []
    mr_old = []
    sf_old = []
    sr_old = []
    number = 0

    if length == 0:
        length = len(experiment.dataset)
    for exp_len in range(length):
        combined_trace =  experiment.get_dataset("traces_p"+str(exp_len)).read_all()
        #reshape the traces as they were captured in segmented fashion 
        combined_trace_reshaped =  np.array(combined_trace[:]).reshape(8*12500,10400)
        #split the traces into random and fixed 
        fixed_traces=combined_trace_reshaped[::2]
        random_traces=combined_trace_reshaped[1::2]

        #this loop calculates ttest trace by trace and saves the intermediate values before calculating the next dataset
        for k in trange(len(random_traces[:])):
            welsh_t, new_mr, new_mf,new_sf,new_sr = running_ttest(new_mf,new_mr,new_sf,new_sr,fixed_traces[k][:10000],random_traces[k][:10000], number)
            number = number+1
    return welsh_t
def extract_random_seg(experiment, length=0):

    if length == 0:
        length = len(experiment.dataset)
    for exp_len in trange(length):
        combined_trace =  experiment.get_dataset("traces_p"+str(exp_len)).read_all()
        #reshape the traces as they were captured in segmented fashion 
        combined_trace_reshaped =  np.array(combined_trace[:]).reshape(8*12500,10400)
        #split the traces into random and fixed 
#         fixed_traces=combined_trace_reshaped[::2]
        if exp_len ==0:
            random_traces = combined_trace_reshaped[1::2]
        else:
            random_traces = np.concatenate((random_traces,combined_trace_reshaped[1::2]),axis =0)
    return random_traces


def extract_random(experiment, length=0):

    if length == 0:
        length = len(experiment.dataset)
    for exp_len in trange(length):
        random_traces =  experiment.get_dataset("random_traces_p"+str(exp_len)).read_all()
        #reshape the traces as they were captured in segmented fashion 
#         combined_trace_reshaped =  np.array(combined_trace[:]).reshape(8*12500,10400)
        #split the traces into random and fixed 
#         fixed_traces=combined_trace_reshaped[::2]
        if exp_len ==0:
            random_trace = random_traces
        else:
            random_trace = np.concatenate((random_traces,random_traces),axis =0)
    return random_trace
    

def ttest_experiment(experiment, length = 0):
   #variables we need 
    fixed_traces = []
    random_traces = []
    welsh_t = []
    new_mr = []
    new_mf =[]
    new_sf = []
    new_sr = []
    mf_old = []
    mr_old = []
    sf_old = []
    sr_old = []
    number = 0

    if length == 0:
        length = len(experiment.dataset)
    for exp_len in range(length):
        fixed_traces =  experiment.get_dataset("fixed_traces_p"+str(exp_len)).read_all()
        random_traces =  experiment.get_dataset("random_traces_p"+str(exp_len)).read_all()

        #this loop calculates ttest trace by trace and saves the intermediate values before calculating the next dataset
        for k in trange(len(random_traces[:])):
            welsh_t, new_mr, new_mf,new_sf,new_sr = running_ttest(new_mf,new_mr,new_sf,new_sr,fixed_traces[k][:],random_traces[k][:], number)
            number = number+1
    return welsh_t


# note we'll have to split this into two groups and run it twice
import numpy as np
from math import sqrt
def running_ttest(mf_old,mr_old,sf_old,sr_old,new_tf, new_tr, n):
    trace_len = len(new_tf)

    if (n==0): 
        new_mf = new_tf
        new_mr = new_tr
        new_sf = new_tf - new_mf
        new_sr = new_tf - new_mf
        welsh_t = new_sf

        return  welsh_t, new_mr, new_mf,new_sf,new_sr
    elif(n>0):

        new_mf = mf_old + (new_tf - mf_old) / (n+1)
        new_mr = mr_old + (new_tr - mr_old) / (n+1)

        new_sf = sf_old + (new_tf - mf_old) * (new_tf - new_mf)
        new_sr = sr_old + (new_tr - mr_old) * (new_tr - new_mr)
        
        new_stdf = np.sqrt(np.array(new_sf/(n)))
        new_stdr = np.sqrt(np.array(new_sr/(n)))

        welsh_t = np.array(new_mf - new_mr)/np.sqrt(np.array((new_stdf**2))/(n+1) + np.array((new_stdr**2))/(n+1))
        return welsh_t, new_mr, new_mf,new_sf,new_sr#, n